In [1]:
import os
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import pycocotools
import tensorflow as tf
from pathlib import Path
print(tf.__version__)

2.6.0


In [2]:
DATA_DIR = Path("./data/20k")
TRAIN_FILE = DATA_DIR / "20k_train.tfrecord"
TEST_FILE = DATA_DIR / "20k_test.tfrecord"
CLASS_LABELS_FILE = DATA_DIR / "classlabels.txt"
CKPT_DIR = Path("./checkpoints") / "ckpts02"
MODEL_EXPORT_DIR = Path("./export") / "02"
CONTINUE_DIR = Path("./export") / "01"
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(MODEL_EXPORT_DIR, exist_ok=True)

In [3]:
with open(CLASS_LABELS_FILE, "r") as f:
    label_map = [s.strip() for s in f.readlines()]
    
label_map_dict = {}
for i, l in enumerate(label_map):
    label_map_dict[i+1] = l

for k, v in label_map_dict.items():
    print(k, ": ", v)

1 :  circle
2 :  diamond
3 :  halfcircle
4 :  heart
5 :  hexagon
6 :  hexstar
7 :  moon
8 :  octagon
9 :  pentagon
10 :  pentstar
11 :  plus
12 :  square
13 :  trapezoid
14 :  triangle


In [4]:
train_data = object_detector.DataLoader(
    tfrecord_file_patten = str(TRAIN_FILE.resolve()),
    size = 18000,
    label_map = label_map_dict
)

test_data = object_detector.DataLoader(
    tfrecord_file_patten = str(TEST_FILE.resolve()),
    size = 2000,
    label_map = label_map_dict
)

In [5]:
hparams = {
    "map_freq" : 1,
    "learning_rate" : 0.05,
    "jitter_max" : 0.5,
    "jitter_min" : 0.05,
    "lr_decay_method" : "constant",
}
spec = object_detector.EfficientDetLite2Spec(
    uri=str(CONTINUE_DIR.resolve()),
    hparams = hparams,
    var_freeze_expr='(fpn_cells|resample_p6)',
)

2021-08-30 14:54:14.344182: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1885] Ignoring visible gpu device (device: 1, name: NVS 510, pci bus id: 0000:03:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.
2021-08-30 14:54:14.345131: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-30 14:54:14.858979: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7409 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


In [6]:
model = object_detector.create(
    train_data = train_data,
    model_spec = spec,
    validation_data = test_data,
    epochs = 500,
    batch_size = 64,
    train_whole_model = False,
    do_train = True,
)

INFO:tensorflow:Retraining the models...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ERROR:absl:hub.KerasLayer is trainable but has zero trainable weights.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ValueError: in user code:

    /home/ms/.pyenv/versions/tflite_model_maker/lib/python3.7/site-packages/tensorflow_hub/keras_layer.py:236 call  *
        result = f()
    /home/ms/.pyenv/versions/tflite_model_maker/lib/python3.7/site-packages/tensorflow/python/saved_model/load.py:664 _call_attribute  **
        return instance.__call__(*args, **kwargs)
    /home/ms/.pyenv/versions/tflite_model_maker/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py:885 __call__
        result = self._call(*args, **kwds)
    /home/ms/.pyenv/versions/tflite_model_maker/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py:933 _call
        self._initialize(args, kwds, add_initializers_to=initializers)
    /home/ms/.pyenv/versions/tflite_model_maker/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py:760 _initialize
        *args, **kwds))
    /home/ms/.pyenv/versions/tflite_model_maker/lib/python3.7/site-packages/tensorflow/python/eager/function.py:3066 _get

In [ ]:
model.evaluate(
    data = test_data,
    batch_size=64,
)